In [87]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.decomposition import PCA, KernelPCA

import os
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.svm import SVC
from load_txt import load_txt
from sklearn.decomposition import PCA, KernelPCA
from ik import IKSimilarity
from sklearn.neighbors import KDTree

import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x,y = make_classification(n_samples = 1000)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y)


# X, y = make_circles(n_samples=1_000, factor=0.3, noise=0.05, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)




In [80]:
import numpy as np
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import KDTree

def IKFeature(data, Sdata=None, psi=64, t=200, Sp=True):
    if Sdata is None:
        Sdata = data.copy()

    sizeS = Sdata.shape[0]
    sizeN = data.shape[0]

    for _ in range(t):
        subIndex = check_random_state(None).choice(sizeS, psi, replace=False)
        
        tdata = Sdata[subIndex, :]
        #distances = pairwise_distances(tdata, data, metric='euclidean')
        #nn_indices = np.argmin(distances, axis=0)
        tree = KDTree(tdata) 
        dist, nn_indices = tree.query(data, k=1)        
        
        OneFeature = np.zeros((sizeN, psi), dtype=int)
        OneFeature[np.arange(sizeN), nn_indices] = 1
        
        if Feature is None:
            Feature = OneFeature  # Initialize Feature with OneFeature in the first iteration
        else:
            Feature = np.hstack((Feature, OneFeature))  # Add OneFeature to Feature vertically

    return Feature  # sparse matrix
 

def IKSimilarity(data, Sdata=None, psi=64, t=200):
    Feature = IKFeature(data, Sdata, psi, t)
    SimMatrix = np.dot(Feature, Feature.T) / t
    return SimMatrix



In [92]:
def Build_tree(Sdata, psi=64, t=200):

    sizeS = Sdata.shape[0]
    Feature = None
    
    tree_list = []
    for _ in range(t):
        subIndex = check_random_state(None).choice(sizeS, psi, replace=False)
        
        tdata = Sdata[subIndex, :]
        #distances = pairwise_distances(tdata, data, metric='euclidean')
        #nn_indices = np.argmin(distances, axis=0)
        tree = KDTree(tdata) 
        
        tree_list.append(tree)
    return tree_list
    
#         dist, nn_indices = tree.query(data, k=1)        
        
#         OneFeature = np.zeros((sizeN, psi), dtype=int)
#         OneFeature[np.arange(sizeN), nn_indices] = 1
        
#         if Feature is None:
#             Feature = OneFeature  # Initialize Feature with OneFeature in the first iteration
#         else:
#             Feature = np.hstack((Feature, OneFeature))  # Add OneFeature to Feature vertically

#     return Feature  # sparse matrix
 

In [94]:
tree_list = Build_tree(x_train)

In [100]:
def IKFeature_tree(data, tree_list, psi=64, t=200):
    sizeN = data.shape[0]
    Feature = None

    for _ in range(t):
        tree = tree_list[_]
        dist, nn_indices = tree.query(data, k=1)        
        
        OneFeature = np.zeros((sizeN, psi), dtype=int)
        OneFeature[np.arange(sizeN), nn_indices] = 1
        
        if Feature is None:
            Feature = OneFeature  # Initialize Feature with OneFeature in the first iteration
        else:
            Feature = np.hstack((Feature, OneFeature))  # Add OneFeature to Feature vertically

    return Feature  # sparse matrix
 


In [103]:
train_feature = IKFeature_tree(x_train, tree_list)

In [104]:
test_feature = IKFeature_tree(x_test, tree_list)

In [108]:
t = 200

In [105]:
train_feature.shape

(750, 12800)

In [106]:
test_feature.shape

(250, 12800)

In [109]:
np.dot(train_feature, train_feature.T) / t

array([[64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       ...,
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.]])

In [ ]:


# Method 2
def get_gram(x1, x2, kernel):
    return np.array([[kernel(_x1, _x2) for _x2 in x2] for _x1 in x1])

def RBF(x1, x2, gamma  = 1):
    return np.exp(-gamma * np.linalg.norm(x1-x2))

clf1 = SVC(kernel = 'precomputed')
clf1.fit(get_gram(x_train, x_train, RBF), y_train)
print(f'Accuracy on Custom Kernel: {accuracy_score(y_test, clf1.predict(get_gram(x_test, x_train, RBF)))}')


1.0

In [89]:
x_train

array([[-0.20859006,  0.98726917, -0.45878911, ..., -1.31777554,
        -0.93826654,  0.90428525],
       [ 0.42733036, -0.14509267, -1.74282177, ...,  1.30932456,
         2.28761128,  0.69444053],
       [ 0.06649308, -0.29983229,  1.40733446, ..., -0.06199133,
         1.07556671,  0.68275835],
       ...,
       [-0.91710569, -0.73271751, -1.66633431, ...,  0.01258449,
         0.83710496, -0.88024936],
       [ 0.02470963, -0.32864367, -0.17992339, ..., -0.43626829,
        -0.06912585,  0.47887312],
       [-0.50776275,  0.01291784,  0.71763245, ..., -1.11798464,
         1.36597444,  1.50631067]])

In [86]:
print("Train and Train",IKFeature(x_train,x_train).shape)
#print("Train and Test",IKFeature(x_train,x_test).shape)
#print("Test and Test",IKFeature(x_test,x_test).shape)
print("Test and Train",IKFeature(x_test,x_train).shape)

Train and Train (750, 12800)
Test and Train (250, 12800)


In [81]:
# Find train Kernal

'''
IKSimilarity <- function(data, Sdata=data, psi = 64, t = 200) {
  Feature<-IKFeature(data, Sdata, psi, t)
  SimMatrix <- Feature%*%t(as.matrix(Feature))/t # the similarity matrix based on Isolation kernel
}
'''

# IKFeature(x_train,x_train) %*% IKFeature(x_train,x_train).T

# (750, 12800)  %*% (12800, 750)  = (750, 750)


(750, 750)

In [82]:
# Find test Kernal (FAILED)

# IKFeature(x_train,x_test) %*% IKFeature(x_train,x_test).T

# (750, 12800)  %*% (12800, 750)  = (750, 750)

(750, 750)

In [ ]:
## Cal similarity matrix manually

In [72]:
train_feature = IKFeature(x_train,x_train)

In [73]:
test_feature = IKFeature(x_test,x_train)

In [74]:
t = 200

In [75]:
## Similarity for train 


# IKFeature(x_train,x_train) %*% IKFeature(x_train,x_train).T

# (750, 12800) %*% (12800,750)


train_sim = np.dot(train_feature, train_feature.T) / t


In [76]:
## Similarity for test 


# IKFeature(x_test,x_train) %*% IKFeature(x_train,x_train).T

# (250, 12800) %*% (12800,750)


test_sim = np.dot(test_feature, train_feature.T) / t


(250, 750)

(750, 750)

In [115]:
class Isolation_Kernal:
    def __init__(self, psi=64, t=200):
        self.t = t  # attribute: t
        self.psi = psi    # attribute: t
        self.tree_list = []

    def build_tree(self,Sdata):
        t = self.t
        psi = self.psi
        
        sizeS = Sdata.shape[0]

        tree_list = []
        for _ in range(t):
            subIndex = check_random_state(None).choice(sizeS, psi, replace=False)

            tdata = Sdata[subIndex, :]
            #distances = pairwise_distances(tdata, data, metric='euclidean')
            #nn_indices = np.argmin(distances, axis=0)
            tree = KDTree(tdata) 

            tree_list.append(tree)
            
        self.tree_list = tree_list
        
        
    def build_feature(self,data):
        t = self.t
        psi = self.psi
        tree_list = self.tree_list
        
        sizeN = data.shape[0]
        Feature = None

        for _ in range(t):
            tree = tree_list[_]
            dist, nn_indices = tree.query(data, k=1)        

            OneFeature = np.zeros((sizeN, psi), dtype=int)
            OneFeature[np.arange(sizeN), nn_indices] = 1

            if Feature is None:
                Feature = OneFeature  # Initialize Feature with OneFeature in the first iteration
            else:
                Feature = np.hstack((Feature, OneFeature))  # Add OneFeature to Feature vertically

        return Feature  # sparse matrix
    
    def cal_similarity(self,feature1,feature2):
        return np.dot(feature1, feature2.T) / self.t

In [117]:
IK = Isolation_Kernal()

In [118]:
IK.build_tree(X_train)

In [119]:
train_feature = IK.build_feature(X_train)

In [120]:
test_feature = IK.build_feature(X_test)

In [121]:
IK.cal_similarity(test_feature,train_feature)

array([[56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565],
       [56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565],
       [56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565],
       ...,
       [56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565],
       [56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565],
       [56.565, 56.565, 56.565, ..., 56.565, 56.565, 56.565]])

In [122]:
IK.cal_similarity(train_feature,train_feature)

array([[64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       ...,
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.],
       [64., 64., 64., ..., 64., 64., 64.]])